Following [this](https://pytorch-lightning.readthedocs.io/en/stable/extensions/callbacks.html)


# Callbacks

A callback is a self-contained program that can be reused across projects.

Lightning has a callback system to execute them when needed. Callbacks should capture **NON-ESSENTIAL** logic that is 
**NOT** required for your lightning module to run.

Here’s the flow of how the callback hooks are executed:

<img src="./assets/callback_overview.png" width="1000"/>

**An overall Lightning system should have:**
* `Trainer` for all engineering
* `LightningModule` for all research code.
* `Callbacks` for non-essential code.

**Example:**

```python
from pytorch_lightning.callbacks import Callback


class MyPrintingCallback(Callback):  # NOTE.
    def on_train_start(self, trainer, pl_module):  # NOTE.
        print("Training is starting")

    def on_train_end(self, trainer, pl_module):  # NOTE.
        print("Training is ending")


trainer = Trainer(callbacks=[MyPrintingCallback()])  # NOTE.
```

We successfully extended functionality without polluting our super clean lightning module research code.

### Examples

Some random examples: https://pytorch-lightning.readthedocs.io/en/stable/extensions/callbacks.html#examples

### Built-in Callbacks

Lightning has a few built-in callbacks.

See list here:

https://pytorch-lightning.readthedocs.io/en/stable/extensions/callbacks.html#built-in-callbacks

Noteworthy:
* [`EarlyStopping`](https://pytorch-lightning.readthedocs.io/en/stable/api/pytorch_lightning.callbacks.EarlyStopping.html#pytorch_lightning.callbacks.EarlyStopping)
* [`RichModelSummary`](https://pytorch-lightning.readthedocs.io/en/stable/api/pytorch_lightning.callbacks.RichModelSummary.html#pytorch_lightning.callbacks.RichModelSummary)
* [`Timer`](https://pytorch-lightning.readthedocs.io/en/stable/api/pytorch_lightning.callbacks.Timer.html#pytorch_lightning.callbacks.Timer)
* [`TQDMProgressBar`](https://pytorch-lightning.readthedocs.io/en/stable/api/pytorch_lightning.callbacks.Timer.html#pytorch_lightning.callbacks.Timer)
* ...

> 🤔 Investigate further:
>
> For a richer collection of callbacks, check out our [bolts library](https://lightning-bolts.readthedocs.io/en/stable/index.html).

### Save Callback state

Some callbacks require *internal state* in order to function properly.

You can optionally choose to persist your callback’s state *as part of model checkpoint* files using `state_dict()` and `load_state_dict()`.

> Note that the returned state **must be able to be pickled**.

When your callback is meant to be used only as a *singleton* callback then implementing the above two hooks is enough to persist state effectively.

However, if passing multiple instances of the callback to the `Trainer` is supported, then the callback must define a `state_key` property in order for Lightning to be able to distinguish the different states when loading the callback state.

This concept is best illustrated by the following example.

In [1]:
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import Callback


class Counter(Callback):
    def __init__(self, what="epochs", verbose=True):
        self.what = what
        self.verbose = verbose
        self.state = {"epochs": 0, "batches": 0}

    @property
    def state_key(self):  # NOTE!
        # note: we do not include `verbose` here on purpose
        return self._generate_state_key(what=self.what)

    def on_train_epoch_end(self, *args, **kwargs):
        if self.what == "epochs":
            self.state["epochs"] += 1

    def on_train_batch_end(self, *args, **kwargs):
        if self.what == "batches":
            self.state["batches"] += 1

    def load_state_dict(self, state_dict):
        self.state.update(state_dict)

    def state_dict(self):
        return self.state.copy()


# two callbacks of the same type are being used
trainer = Trainer(callbacks=[Counter(what="epochs"), Counter(what="batches")])  # NOTE 2 same callbacks passed!

GPU available: True (cuda), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/mnt/data-linux/miniconda3/envs/learn_py38_lightning/lib/python3.8/site-packages/pytorch_lightning/trainer/setup.py:175: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(


A Lightning checkpoint from this Trainer with the two stateful callbacks will include the following information:

```python
{
    "state_dict": ...,
    "callbacks": {
        "Counter{'what': 'batches'}": {"batches": 32, "epochs": 0},
        "Counter{'what': 'epochs'}": {"batches": 0, "epochs": 2},
        ...
    }
}
```

The implementation of a `state_key` is essential here. If it were missing, Lightning would not be able to disambiguate
the state for these two callbacks, and `state_key` by default only defines the class name as the key, e.g., here `Counter`.

### Best Practices

The following are best practices when using/designing callbacks.
1. Callbacks should be isolated in their functionality.
1. Your callback should not rely on the behavior of other callbacks in order to work properly.
1. Do not manually call methods from the callback.
1. Directly calling methods (eg. `on_validation_end`) is strongly discouraged.
1. Whenever possible, your callbacks should not depend on the order in which they are executed.


### Entry Points

See: https://pytorch-lightning.readthedocs.io/en/stable/extensions/callbacks.html#entry-points

### ℹ️ Callback API

The `Callback` class is the base for all the callbacks in Lightning just like the
`LightningModule` is the base for all models.

It defines a public interface that each callback implementation must follow.

For a summary of callback interface, see: https://pytorch-lightning.readthedocs.io/en/stable/extensions/callbacks.html#callback-api

**Notable**
* `on_fit_{start,end}`
* `on_train_{start,end}`
* `on_{train,test,validation,predict}_{epoch,batch}_{start,end}`
* `on_predict_{start,end}`
* `on_save_checkpoint`
* ...